In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from scipy import integrate
import seaborn as sns
import itertools
from matplotlib.lines import Line2D
import matplotlib.pylab as plb
from sklearn.linear_model import LinearRegression
import matplotlib.style as style
from matplotlib.patches import Patch

#data_path = "/Users/Broccolee/Google Drive/Grad School/TU BERLIN/Negative Emissions & Carbon Markets/Data/"
#plot_path = "/Users/Broccolee/Google Drive/Grad School/TU BERLIN/Negative Emissions & Carbon Markets/Plots/"
data_path = "/Users/kaylinlee/Desktop/KL_ISIpedia/Thesis/Code/Data/"
plot_path = "/Users/kaylinlee/Desktop/KL_ISIpedia/Thesis/Code/Plots/"

pd.set_option('display.max_columns', 500)

In [5]:
BECCS_leakage=pd.read_excel(data_path+'netzero_all_models_NEW.xls')
BECCS_transferred_allscens=pd.read_excel(data_path + 'BECCS_transferred_allscens.xls')

In [6]:
scen_allmodels = ['SSP1-19-SPA1-V17', 'SSP2-19-SPA2-V17', 'SSP1-26-SPA1-V17',
       'SSP2-26-SPA2-V17', 'SSP5-19-SPA5-V17', 'SSP5-26-SPA5-V17',
       'SSP1-19-SPA0-V17', 'SSP2-LF-SPA2-V17']
scen_15_lo = ['SSP1-19-SPA0-V17'] #only no / limited overshoot
scen_15_ho = ['SSP2-LF-SPA2-V17', 'SSP1-19-SPA1-V17', 'SSP5-19-SPA5-V17','SSP2-19-SPA2-V17'] # 1.5 high overshoot
scen_15_all= ['SSP1-19-SPA1-V17', 'SSP2-19-SPA2-V17', 'SSP5-19-SPA5-V17',
       'SSP1-19-SPA0-V17','SSP2-LF-SPA2-V17'] #all 1.5 scens
scen_2 = ['SSP2-26-SPA2-V17','SSP1-26-SPA1-V17', 'SSP5-26-SPA5-V17'] #2° scens

In [7]:
BECCS_leakage.head(10)

,Unnamed: 0,MODEL,Region,Scenario,Year,Total_CO2,Total_GHG,BECCS,LANDCDR,TOTAL_CDR,CO2_Emissions,CUM_CO2,CUM_CDR,CUM_BECCS,Unit_CO2,BECCS_Investments,CUM_BECCS_Investments,Unit_Investment,Carbon_Price,Population,GDP,Unit_Population,cumPopulation,2016_emissions,em_change,Unit_GDP,GDP_cap,GDP_cap_avg,GDP_cap_CoI_nan,GDP_cap_CoI,GDP_CoI_nan,GDP_CoI,GDP_CoI_frac,CDR_fair_share_by_GDP,BECCS_fair_share_by_GDP,BECCS_investments_by_GDP,CUM_CDR_fair_share_by_GDP,CUM_BECCS_fair_share_by_GDP,CUM_BECCS_investments_by_GDP,Global_CUM_CO2_by_CUM_POP,Fair_share_by_CUM_POP,Excess_emissions_by_CUM_POP,Exc_em_fac_by_CUM_POP,CDR_fair_share_by_CUM_POP,BECCS_fair_share_by_CUM_POP,CUM_CDR_fair_share_by_CUM_POP,CUM_BECCS_fair_share_by_CUM_POP,CDR_model_vs_fair_share_by_CUM_POP,CDR_model_vs_fair_share_by_GDP,CUM_CDR_model_vs_fair_share_by_CUM_POP,CUM_CDR_model_vs_fair_share_by_GDP,BECCS_Investments_by_POP,CUM_BECCS_Investments_by_POP,BECCS_model_vs_fair_share_by_CUM_POP,BECCS_model_vs_fair_share_by_GDP,BECCS_invest_model_vs_FairShare_by_POP,BECCS_invest_model_vs_FairShare_by_GDP,CUM_BECCS_invest_model_vs_FairShare_by_POP,CUM_BECCS_invest_model_vs_FairShare_by_GDP,CUM_BECCS_model_vs_fair_share_by_CUM_POP,CUM_BECCS_model_vs_fair_share_by_GDP,5%_leakage:AP,10%_leakage:AP,15%_leakage:AP,20%_leakage:AP,25%_leakage:AP,5%_leakage:CPCE,10%_leakage:CPCE,15%_leakage:CPCE,20%_leakage:CPCE,25%_leakage:CPCE
0,0,REMIND,AFR,SSP1-19-SPA1-V17,2020,1215.4939,3233.0,0.0060,222.6215,222.6275,3455.6275,98289.528461,222.6275,0.0060,Mt/yr,0.0013,0.0013,billion US$2005/yr,1.000,996.7,2288000.0,Mio,21023.288404,3127.643169,1.104866,USD Million,2295.575399,13416.412568,NaN,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.797535,142906.547734,-44617.019272,NaN,0.0,0.0,0.0,0.0,-222.6275,-222.627500,-222.6275,-222.627500,0.0,0.0,-0.0060,-0.006000,-0.0013,-0.001300,-0.0013,-0.001300,-0.0060,-0.006000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,REMIND,AFR,SSP1-19-SPA1-V17,2030,1212.0330,2952.0,30.8600,77.8637,108.7237,3060.7237,130871.284461,1879.3835,154.3360,Mt/yr,0.1373,0.6943,billion US$2005/yr,1.145,1181.0,4293000.0,Mio,31911.788404,3127.643169,0.978604,USD Million,3635.055038,18619.142572,NaN,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.475300,206638.395114,-75767.110652,NaN,0.0,0.0,0.0,0.0,-108.7237,-108.723700,-1879.3835,-1879.383500,0.0,0.0,-30.8600,-30.860000,-0.1373,-0.137300,-0.6930,-0.693000,-154.3300,-154.330000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,2,REMIND,AFR,SSP1-19-SPA1-V17,2040,1618.5552,3692.0,352.2000,21.3519,373.5519,4065.5519,166502.662461,4290.7615,2069.6360,Mt/yr,0.0010,1.3858,billion US$2005/yr,1.865,1351.0,8860000.0,Mio,44571.788404,3127.643169,1.299877,USD Million,6558.105107,25680.047933,NaN,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.933099,264448.849359,-97946.186898,NaN,0.0,0.0,0.0,0.0,-373.5519,-373.551900,-4290.7615,-4290.761500,0.0,0.0,-352.2000,-352.200000,-0.0010,-0.001000,-1.3845,-1.384500,-2069.6300,-2069.630000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,3,REMIND,AFR,SSP1-19-SPA1-V17,2050,388.3696,2444.0,949.0000,28.5074,977.5074,3421.5074,203937.958961,11046.0580,8575.6360,Mt/yr,0.0000,1.3908,billion US$2005/yr,3.038,1488.0,17450000.0,Mio,58766.788404,3127.643169,1.093957,USD Million,11727.150538,33297.960146,NaN,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.380471,316193.006837,-112255.047876,NaN,0.0,0.0,0.0,0.0,-977.5074,-977.507400,-11046.0580,-11046.058000,0.0,0.0,-949.0000,-949.000000,0.0000,0.000000,-1.3895,-1.389500,-8575.6300,-8575.630000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,4,REMIND,AFR,SSP1-19-SPA1-V17,2060,-814.2021,738.9,1144.0000,43.1143,1187.1143,1926.0143,230675.567461,21869.1665,19040.6360,Mt/yr,0.0000,1.3908,billion US$2005/yr,4.949,1589.0,30720000.0,Mio,74151.788404,3127.64316

In [4]:
leakage_amount=['5%_leakage:AP','10%_leakage:AP','15%_leakage:AP','20%_leakage:AP','25%_leakage:AP',
           '5%_leakage:CPCE','10%_leakage:CPCE','15%_leakage:CPCE','20%_leakage:CPCE','25%_leakage:CPCE']
trade_amount=['BECCS_model_vs_fair_share_by_CUM_POP','BECCS_model_vs_fair_share_by_GDP']
BECCS_leakage.loc[BECCS_leakage.Year==(2100)&]

,Unnamed: 0,MODEL,Region,Scenario,Year,Total_GHG,TOTAL_CDR,BECCS,CO2_Emissions,CDR_model_vs_fair_share_by_CUM_POP,CDR_model_vs_fair_share_by_GDP,BECCS_model_vs_fair_share_by_GDP,BECCS_model_vs_fair_share_by_CUM_POP,BECCS_leakage_10%:CUM_POP,BECCS_leakage_20%:CUM_POP,BECCS_leakage_30%:CUM_POP,BECCS_leakage_40%:CUM_POP,Total_CDR-10%leakage:CUM_POP,Total_CDR-20%leakage:CUM_POP,Total_CDR-30%leakage:CUM_POP,Total_CDR-40%leakage:CUM_POP,BECCS_leakage_10%:GDP,BECCS_leakage_20%:GDP,BECCS_leakage_30%:GDP,BECCS_leakage_40%:GDP,Total_CDR-10%leakage:GDP,Total_CDR-20%leakage:GDP,Total_CDR-30%leakage:GDP,Total_CDR-40%leakage:GDP,CUM_BECCS_leakage_10%:CUM_POP,CUM_BECCS_leakage_20%:CUM_POP,CUM_BECCS_leakage_30%:CUM_POP,CUM_BECCS_leakage_40%:CUM_POP,CUM_BECCS_leakage_10%:GDP,CUM_BECCS_leakage_20%:GDP,CUM_BECCS_leakage_30%:GDP,CUM_BECCS_leakage_40%:GDP
0,99,REMIND,World,SSP1-19-SPA1-V17,2020,49470.0,690.6849,0.0598,50160.6849,-690.6849,NaN,NaN,-0.0598,0.003237,0.006475,0.009712,0.012949,690.681663,690.678425,690.675188,690.671951,0.003462,0.006925,0.010387,0.013849,690.681438,690.677975,690.674513,690.671051,0.003237,0.006475,0.009712,0.012949,0.003462,0.006925,0.010387,0.013849
1,100,REMIND,World,SSP1-19-SPA1-V17,2030,37090.0,767.0871,424.2000,37857.0871,-767.0871,NaN,NaN,-424.2000,21.016849,42.033699,63.050548,84.067398,746.070251,725.053401,704.036552,683.019702,16.008800,32.017600,48.026400,64.035200,751.078300,735.069500,719.060700,703.051900,105.100433,210.200867,315.301300,420.401734,80.061312,160.122623,240.183935,320.245246
2,101,REMIND,World,SSP1-19-SPA1-V17,2040,25340.0,3972.0767,3788.0000,29312.0767,-3972.0767,NaN,NaN,-3788.0000,193.773709,387.547419,581.321128,775.094837,3778.302991,3584.529281,3390.755572,3196.981863,162.260000,324.520000,486.780000,649.040000,3809.816700,3647.556700,3485.296700,3323.036700,1179.053227,2358.106453,3537.159680,4716.212907,971.405312,1942.810623,2914.215935,3885.621246
3,102,REMIND,World,SSP1-19-SPA1-V17,2050,11810.0,9838.0093,9330.0000,21648.0093,-9838.0093,NaN,NaN,-9330.0000,478.725294,957.450589,1436.175883,1914.901177,9359.284006,8880.558711,8401.833417,7923.108123,333.410000,666.820000,1000.230000,1333.640000,9504.599300,9171.189300,8837.779300,8504.369300,4541.548245,9083.096490,13624.644734,18166.192979,3449.755312,6899.510623,10349.265935,13799.021246
4,103,REMIND,World,SSP1-19-SPA1-V17,2060,854.9,12008.1735,10940.0000,12863.0735,-12008.1735,NaN,NaN,-10940.0000,541.114154,1082.228307,1623.342461,2164.456614,11467.059346,10925.945193,10384.831039,9843.716886,411.200000,822.400000,1233.600000,1644.800000,11596.973500,11185.773500,10774.573500,10363.373500,9640.745485,19281.490969,28922.236454,38562.981938,7172.805312,14345.610623,21518.415935,28691.221246
5,104,REMIND,World,SSP1-19-SPA1-V17,2070,-3084.0,12862.3824,11310.0000,9778.3824,-12862.3824,NaN,NaN,-11310.0000,532.829850,1065.659700,1598.489550,2131.319400,12329.552550,11796.722700,11263.892850,10731.063000,422.800000,845.600000,1268.400000,1691.200000,12439.582400,12016.782400,11593.982400,11171.182400,15010.465502,30020.931004,45031.396506,60041.862007,11342.805312,22685.610623,34028.415935,45371.221246
6,105,REMIND,World,SSP1-19-SPA1-V17,2080,-5499.0,13872.9505,11630.0000,8373.9505,-13872.9505,NaN,NaN,-11630.0000,519.918646,1039.837291,1559.755937,2079.674582,13353.031854,12833.113209,12313.194563,11793.275918,427.330000,854.660000,1281.990000,1709.320000,13445.620500,13018.290500,12590.960500,12163.630500,20274.207979,40548.415958,60822.623937,81096.831916,15593.455312,31186.910623,46780.365935,62373.821246
7,106,REMIND,World,SSP1-19-SPA1-V17,2090,-7517.0,14922.9724,12030.0000,7405.9724,-14922.9724,NaN,NaN,-12030.0000,511.826893,1023.653787,1535.480680,2047.307573,14411.145507,13899.318613,13387.491720,12875.664827,439.280000,878.560000,1317.840000,1757.120000,14483.692400,14044.412400,13605.132400,13165.852400,25432.935673,50865.871347,76298.807020,101731.742693,19926.505312,39853.010623,59779.515935,79706.021246
8,107,REMIND,World,

In [ ]:
BECCS_leakage_allmodels.loc[BECCS_leakage_allmodels.Year==(2100)][]